# A numeric approach to the Monty Hall problem

The Monty Hall problem is famous in the world of probability


> You're playing a gameshow, and the host asks you to choose one of three doors.
>
> Behind one door is a car, and the other two doors have goats. You win whatever you choose!
>
> Once you have chosen a door, the host
> 1. Removes one of doors that you didn't choose (but he will never remove the car), then
> 2. Gives you the option to change your choice of door.
>
> Should you change? Does it matter?

Let's set up the problem

In [1]:
import random

GOAT = 'goat'
CAR = 'car'
DOORS = (0, 1, 2)
AVAILABLE_PRIZES = (GOAT, GOAT, CAR)

def make_prizes():
    return random.sample(AVAILABLE_PRIZES, 3)

prizes = make_prizes()
prizes

['goat', 'car', 'goat']

Now we have the prizes behind each door set.

So let's choose a door.

In [2]:
def choose_door():
    return random.choice(DOORS)

chosen_door = choose_door()
chosen_door

0

Now the host will remove a door

In [3]:
def is_removable(door, chosen_door, prizes):
    '''Remember, the host isn't allowed to remove the door 
    with the car behind it.'''
    return door is not chosen_door \
       and prizes[door] is not CAR

def find_removable_doors(chosen_door, prizes):
    ''' 
    There's something interesting here 
    
    When player chooses the car first go, 
        the host has two goats he could remove.
        
    However, when the player chooses a goat first go,
        the host has no choice, he *must* remove the other goat
    '''
    return [
        door for door in DOORS
        if is_removable(door, chosen_door, prizes)
    ]

def choose_door_to_remove(chosen_door, prizes):
    '''
    So the host's random choice of removable doors isn't always random.
    One third of the time (when the player was right first go), the host
    removes the only door he's allowed to remove.
    '''
    removable_doors = find_removable_doors(chosen_door, prizes)
    return random.choice(removable_doors)

def remaining(chosen_door, removed_door):
    return next(door for door in DOORS
                if door is not chosen_door
                and door is not removed_door)

removed_door = choose_door_to_remove(chosen_door, prizes)
remaining_door = remaining(chosen_door, removed_door)
remaining_door

1

So now you have the choice.

Do you change your mind?

In [4]:
def you_won(door, prizes):
    return prizes[door] is CAR

def announce_outcome(door, prizes):
    if you_won(door, prizes):
        print(f'Congrats, you won a {CAR}!')
    else:
        print(f'Looks like you won a {GOAT}.')

In [5]:
print('Did you change your mind?')
final_chosen_door = chosen_door
announce_outcome(final_chosen_door, prizes)

Did you change your mind?
Looks like you won a goat.


In [6]:
print('Or did you stay with your first choice?')
final_chosen_door = remaining_door
announce_outcome(final_chosen_door, prizes)

Or did you stay with your first choice?
Congrats, you won a car!


What if we want to try a thousand times?  
First, let's make a function to play the whole game

In [7]:
def play_monty_hall(change):
    prizes = make_prizes()
    chosen_door = choose_door()
    removed_door = choose_door_to_remove(chosen_door, prizes)
    alternative_door = remaining(chosen_door, removed_door)
    chosen_door = alternative_door if change else chosen_door
    return you_won(chosen_door, prizes)

won_without_change = play_monty_hall(change=False)
won_with_change = play_monty_hall(change=True)

print('Won with change?', won_with_change)
print('Won without change?', won_without_change)

Won with change? False
Won without change? False


Time to test the strategies

In [8]:
n_trials = 10000
wins_without_change = sum(play_monty_hall(change=False) for _ in range(n_trials))
print(f'Wins without change {wins_without_change}, {100*wins_without_change/n_trials}%')

Wins without change 3422, 34.22%


In [9]:
wins_with_change = sum(play_monty_hall(change=True) for _ in range(n_trials))
print(f'Wins with change {wins_with_change}, {100*wins_with_change/n_trials}%')

Wins with change 6696, 66.96%


So it seems like changing is a much better strategy!

Here's the final code

In [10]:
import random


GOAT = 'goat'
CAR = 'car'
DOORS = (0, 1, 2)
AVAILABLE_PRIZES = (GOAT, GOAT, CAR)


def make_prizes():
    return random.sample(AVAILABLE_PRIZES, 3)

def choose_door():
    return random.choice(DOORS)

def is_removable(door, chosen_door, prizes):
    return door is not chosen_door \
       and prizes[door] is not CAR

def choose_door_to_remove(chosen_door, prizes):
    removable_doors = [door for door in DOORS if is_removable(door, chosen_door, prizes)]
    return random.choice(removable_doors)

def remaining(chosen_door, removed_door):
    return next(door for door in DOORS
                if door is not chosen_door
                and door is not removed_door)

def you_won(door, prizes):
    return prizes[door] is CAR

def play_monty_hall(change):
    prizes = make_prizes()
    chosen_door = choose_door()
    removed_door = choose_door_to_remove(chosen_door, prizes)
    alternative_door = remaining(chosen_door, removed_door)
    chosen_door = alternative_door if change else chosen_door
    return you_won(chosen_door, prizes)


def compare_strategies(n_trials=1000):
    wins_without_change = sum(play_monty_hall(change=False) for _ in range(n_trials))
    wins_with_change = sum(play_monty_hall(change=True) for _ in range(n_trials))
    return (
        f'Wins without change {wins_without_change}, {100*wins_without_change/n_trials}%'
        '\n'
        f'Wins with change {wins_with_change}, {100*wins_with_change/n_trials}%'
    )
    
print(compare_strategies(100000))

Wins without change 33304, 33.304%
Wins with change 66540, 66.54%
